# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
WeatherPy_df = pd.read_csv("../WeatherPy/Output/WeatherPy_Output.csv")
WeatherPy_df


,City Name,City ID,Country Code,Longitude,Latitude,Temp,Feels Like Temp,Min Temp,Max Temp,Pressure,Humidity,Visibility,Wind Speed,Wind Temp,Sunrise,Sunset,Cloudiness
0,Ushuaia,3833367,AR,-68.3000,-54.8000,2.81,-1.20,2.81,2.81,1025,75,10000,4.63,270,1647599425,1647643927,20
1,Avarua,4035715,CK,-159.7750,-21.2078,29.03,32.79,29.03,29.03,1013,70,10000,3.09,40,1647621738,1647665513,80
2,Longyearbyen,2729907,SJ,15.6401,78.2186,-2.09,-7.92,-2.09,-2.09,991,64,10000,5.66,180,1647666022,1647709810,40
3,Voh,2137748,NC,164.7000,-20.9667,25.95,25.95,25.95,25.95,1005,87,10000,0.79,288,1647630267,1647674033,100
4,Phibun Mangsahan,1607730,TH,105.2291,15.2447,26.10,26.10,24.99,28.39,1009,51,10000,1.31,5,1647644658,1647688181,96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,Adrar,2381972,MR,-10.0711,20.5022,17.95,16.69,17.95,17.95,1011,34,10000,3.95,40,1647672324,1647715848,64
576,Ukiah,5404476,US,-123.2078,39.1502,12.60,11.33,9.80,15.36,1017,54,10000,2.57,20,1647613157,1647656545,0
577,Ushtobe,1517637,KZ,77.9838,45.2514,0.09,-2.90,0.09,0.09,1033,65,10000,2.48,275,1647651234,1647694680,100
578,Tigil',2120612,RU,158.6667,57.8000,-0.31,-6.15,-0.31,-0.31,1026,73,10000,6.64,140,1647631909,1647675285,73


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# House the city coordinates in a variable.

coordinates = WeatherPy_df[["Latitude", "Longitude"]]

# House the humidity  values in a variable.

humidity = WeatherPy_df["Humidity"]

In [4]:
# Create Heatmap

# Set the output image size:

figure_layout = {'width': '900px','height': '600px','border': '1px solid black','padding': '1px','margin': '0 auto 0 auto'}

fig = gmaps.figure(layout = figure_layout, zoom_level=2,center = (15,25))

# Create heat layer

heat_layer = gmaps.heatmap_layer(coordinates, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 3)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Convert the units in the ideal weather conditions to METRIC:

# A max temperature lower than 26.67 degrees Celcius (80 degrees F), but higher than 21.1 degrees Celcius (70 grees F).
# The max wind speed is 4.4704 m/s (10 miles per hour).

# Filter by max ideal temperature:

filter1_df = WeatherPy_df.loc[WeatherPy_df["Temp"] < 26.67, :]

# Filter by min ideal temperature:

filter2_df = filter1_df.loc[filter1_df["Temp"] > 21.1, :]

# Filter by max wind speed:

filter3_df = filter2_df.loc[filter2_df["Wind Speed"] < 4.4704, :]

# Filter by max wind speed:

filter4_df = filter3_df.loc[filter3_df["Cloudiness"] == 0, :]
ideal_weather_df = filter4_df.dropna()
ideal_weather_df

,City Name,City ID,Country Code,Longitude,Latitude,Temp,Feels Like Temp,Min Temp,Max Temp,Pressure,Humidity,Visibility,Wind Speed,Wind Temp,Sunrise,Sunset,Cloudiness
48,Ibrā’,287832,OM,58.5334,22.6906,22.81,22.16,22.81,22.81,1011,39,10000,1.63,340,1647655872,1647699376,0
57,Bandarbeyla,64814,SO,50.8122,9.4942,24.49,24.90,24.49,24.49,1013,73,10000,4.19,38,1647657700,1647701253,0
77,Cabo San Lucas,3985710,MX,-109.9124,22.8909,23.06,23.25,20.10,23.06,1007,70,10000,1.10,287,1647609931,1647653391,0
349,Black River,3491355,JM,-77.8487,18.0264,23.50,23.86,23.50,23.50,1017,75,10000,2.25,72,1647602227,1647645708,0
374,Panzhihua,6929460,CN,101.7128,26.5851,25.61,24.75,25.61,25.61,1007,20,10000,0.20,137,1647645523,1647689004,0
441,La Romana,3500957,DO,-68.9728,18.4273,22.98,23.53,22.98,22.98,1018,84,10000,4.24,54,1647600099,1647643576,0
522,Guiratinga,3461733,BR,-53.7583,-16.3494,22.00,22.61,22.00,22.00,1013,90,10000,0.49,123,1647596315,1647640059,0
551,Miramar,4164601,US,-80.2323,25.9873,24.91,25.73,23.33,26.11,1019,87,10000,1.34,119,1647602821,1647646258,0
570,Ovalle,3877918,CL,-71.2003,-30.5983,22.92,22.49,22.92,22.92,1013,47,10000,0.52,227,1647686851,1647730656,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=ideal_weather_df
hotel_df["Hotel Name"] = ""

In [7]:
len(hotel_df)

9

In [9]:
# Create a list to house the hotel names generated by the request.

Hotel_Names=[]  

# Store cities and their latitudes & logitudes (from hotel_df) as variables.

city = hotel_df["City Name"].tolist()
latitude = hotel_df['Latitude'].tolist()
longitude = hotel_df['Longitude'].tolist()


# Build URL and make Google Places API request.

for i in range(len(hotel_df)):
    coordinates = f"{latitude[i]}, {longitude[i]}"

    # Specify the parameters.
    
    params = {"location": coordinates, "types": "hotel", "keyword": "lodging", "radius": 5000, "key": g_key}
    
    # Specify the url that will be used as the basis of the request.
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    try:
        
        # Run the request and store the name of the hotel that is first returned.
        
        response = requests.get(url, params=params).json()
        Hotel_Names.append(response['results'][0]['name'])
    
    except:
        
        # If no data exists then add the following string to the hotel names list.
        
        Hotel_Names.append('NO HOTEL FOUND')
        
# Once the iterations are finished, add the hotel name list as a column in hotels_df.

hotel_df['Hotel Name'] = Hotel_Names 
hotel_df[["City Name", "Longitude", "Latitude", "Hotel Name", "Temp", "Cloudiness", "Wind Speed"]]

,City Name,Longitude,Latitude,Hotel Name,Temp,Cloudiness,Wind Speed
48,Ibrā’,58.5334,22.6906,المستقبل للشقق الفندقية,22.81,0,1.63
57,Bandarbeyla,50.8122,9.4942,JABIR HOTEL,24.49,0,4.19
77,Cabo San Lucas,-109.9124,22.8909,Grand Solmar Land's End Resort & Spa,23.06,0,1.10
349,Black River,-77.8487,18.0264,Idlers' Rest Beach Hotel,23.50,0,2.25
374,Panzhihua,101.7128,26.5851,Chuanhui Grand Hotel,25.61,0,0.20
441,La Romana,-68.9728,18.4273,Casa de Campo Resort and Villas,22.98,0,4.24
522,Guiratinga,-53.7583,-16.3494,Hotel Angra do Cerrado - Guiratinga MT,22.00,0,0.49
551,Miramar,-80.2323,25.9873,Stadium Hotel,24.91,0,1.34
570,Ovalle,-71.2003,-30.5983,Apart Hotel Dolce Vita,22.92,0,0.52


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country Code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [16]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…